In [183]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split

In [184]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [185]:
train_y = train[["Transported"]].astype(int)
test_y = test[["PassengerId"]]

In [186]:

train['Group'] = train['PassengerId'].apply(lambda x: x.split('_')[0])
train['GroupSize'] = train.groupby('Group')['PassengerId'].transform('count')
train = train.drop('Group', axis=1)

test['Group'] = test['PassengerId'].apply(lambda x: x.split('_')[0])
test['GroupSize'] = test.groupby('Group')['PassengerId'].transform('count')
test = test.drop('Group', axis=1)

def get_cabin_side(cabin):
    if pd.isna(cabin):  
        return None  
    side = cabin.split('/')[-1]
    if side == 'P':
        return 0
    elif side == 'S':
        return 1
    else:
        return None

train['CabinSide'] = train['Cabin'].apply(get_cabin_side)
test['CabinSide'] = test['Cabin'].apply(get_cabin_side)

def get_cabin_deck(cabin):
    if pd.isna(cabin): 
        return None 
    try:
        parts = cabin.split('/')
        return f"{parts[0]}"
    except:
        return None

train['CabinDeck'] = train['Cabin'].apply(get_cabin_deck)
test['CabinDeck'] = test['Cabin'].apply(get_cabin_deck)



In [187]:
train['VIP'] = train['VIP'].fillna(0)
train["VIP"] = train["VIP"].astype(int)

train['CryoSleep'] = train['CryoSleep'].fillna(0)
train["CryoSleep"] = train["CryoSleep"].astype(int)

test['VIP'] = test['VIP'].fillna(0)
test["VIP"] = test["VIP"].astype(int)

test['CryoSleep'] = test['CryoSleep'].fillna(0)
test["CryoSleep"] = test["CryoSleep"].astype(int)

train = train.drop(columns=["Name", "PassengerId", "Transported", "Cabin"])
test = test.drop(columns=["Name", "PassengerId", "Cabin"])


In [188]:
train[["RoomService_log"]] = np.log10(1 + train[["RoomService"]])
train[["FoodCourt_log"]] = np.log10(1 + train[["FoodCourt"]])
train[["ShoppingMall_log"]] = np.log10(1 + train[["ShoppingMall"]])
train[["Spa_log"]] = np.log10(1 + train[["Spa"]])
train[["VRDeck_log"]] = np.log10(1 + train[["VRDeck"]])

test[["RoomService_log"]] = np.log10(1 + test[["RoomService"]])
test[["FoodCourt_log"]] = np.log10(1 + test[["FoodCourt"]])
test[["ShoppingMall_log"]] = np.log10(1 + test[["ShoppingMall"]])
test[["Spa_log"]] = np.log10(1 + test[["Spa"]])
test[["VRDeck_log"]] = np.log10(1 + test[["VRDeck"]])

train = train.drop(["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], axis=1)
test = test.drop(["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], axis=1)


In [189]:
num_cols = [cname for cname in train.columns if train[cname].dtype in ["int64", "float64"]]
cat_cols = [cname for cname in train.columns if train[cname].dtype == "object"]

In [190]:
from sklearn.pipeline import Pipeline
num_transformer = Pipeline(steps=[("scaler", StandardScaler())])

cat_transformer = Pipeline(
    steps=[
        (
            "onehot",
            OneHotEncoder(drop='if_binary', handle_unknown="ignore")
        )
    ]
)

In [191]:
ct = ColumnTransformer(
    transformers = [
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols),
    ],
    remainder="passthrough"
)

In [192]:
train = ct.fit_transform(train)
test = ct.fit_transform(test)


In [193]:
print(train.shape, test.shape)

(8693, 27) (4277, 27)


In [194]:
classifiers = {
    "LGBM": LGBMClassifier(
        learning_rate=0.01, max_depth=8, n_estimators=500, random_state=0, num_leaves=31
    ),
}

In [195]:

FOLDS = 5
y = train_y.values

In [196]:

X_train_opt, X_test_opt, y_train_opt, y_test_opt = train_test_split(train, train_y, test_size=0.2, random_state=42)

def optimize_lgbm(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
        'max_depth': trial.suggest_int('max_depth', 4, 14),
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'random_state': 0
    }

    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(X_train_opt, y_train_opt)

    y_pred = lgb_model.predict(X_test_opt)
    accuracy = accuracy_score(y_test_opt, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(optimize_lgbm, n_trials=100)

print('Лучшие гиперпараметры:', study.best_params)
print('Лучшая оценка:', study.best_value)

classifiers = {
    "LGBM": lgb.LGBMClassifier(
        learning_rate=study.best_params['learning_rate'],
        max_depth=study.best_params['max_depth'],
        n_estimators=study.best_params['n_estimators'],
        num_leaves=study.best_params['num_leaves'],
        random_state=0
    ),
}

[I 2025-03-21 20:41:19,788] A new study created in memory with name: no-name-f2505ed4-6964-4454-9adf-741ce16492c4


[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1406
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


C:\Users\ivank\AppData\Local\Temp\ipykernel_16620\2925098800.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\ivank\OneDr

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000283 seconds.
Yo

C:\Users\ivank\AppData\Local\Temp\ipykernel_16620\2925098800.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\ivank\OneDr

...

[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1406
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-03-21 20:41:47,896] Trial 98 finished with value: 0.7941345600920069 and parameters: {'learning_rate': 0.01647570904639744, 'max_depth': 13, 'n_estimators': 493, 'num_leaves': 18}. Best is trial 60 with value: 0.79700977573318.
C:\Users\ivank\AppData\Local\Temp\ipykernel_16620\2925098800.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expe

[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1406
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-03-21 20:41:48,148] Trial 99 finished with value: 0.7912593444508338 and parameters: {'learning_rate': 0.017885192409074525, 'max_depth': 5, 'n_estimators': 476, 'num_leaves': 14}. Best is trial 60 with value: 0.79700977573318.


Лучшие гиперпараметры: {'learning_rate': 0.02334934150784144, 'max_depth': 12, 'n_estimators': 467, 'num_leaves': 11}
Лучшая оценка: 0.79700977573318


In [197]:
preds = np.zeros(test.shape[0])
preds_train = np.zeros(train.shape[0])

In [198]:
for key, classifier in classifiers.items():
    start = time.time()

    cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)

    score = 0
    for fold, (train_idx, val_idx) in enumerate(cv.split(train, y)):
        X_train, X_valid = train[train_idx], train[val_idx]
        y_train, y_valid = y[train_idx], y[val_idx]

        clf = classifier
        clf.fit(X_train, y_train)

        preds += clf.predict_proba(test)[:, 1]
        preds_train += clf.predict_proba(train)[:, 1]
        score += clf.score(X_valid, y_valid)
        print("Average infold validation accuracy: ", key, fold, np.round(100 * clf.score(X_valid, y_valid), 2))
        
    score = score / FOLDS
    stop = time.time()
    print(np.round(100 * score, 2))
    print(np.round(stop - start) / 60, 2)



c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1406
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380


c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\pr

Average infold validation accuracy:  LGBM 0 80.97
[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1406
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380


c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\pr

...

c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\pr

Average infold validation accuracy:  LGBM 2 81.14
[LightGBM] [Info] Number of positive: 3503, number of negative: 3452
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000306 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1407
[LightGBM] [Info] Number of data points in the train set: 6955, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503666 -> initscore=0.014666
[LightGBM] [Info] Start training from score 0.014666


c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\pr

Average infold validation accuracy:  LGBM 3 80.03
[LightGBM] [Info] Number of positive: 3503, number of negative: 3452
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1407
[LightGBM] [Info] Number of data points in the train set: 6955, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503666 -> initscore=0.014666
[LightGBM] [Info] Start training from score 0.014666
Average infold validation accuracy:  LGBM 4 81.36
80.84
0.016666666666666666 2


c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ivank\OneDrive\Desktop\ML_module_1\notebooks\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [199]:
preds = preds / (FOLDS * len(classifiers))
preds_train = preds_train / (FOLDS * len(classifiers))

In [200]:
output = pd.DataFrame(
    {
        "PassengerId": test_y["PassengerId"],
        "Transported": (preds > 0.5)
    }
)
output.to_csv("submission.csv", index=False)